In [24]:
import jsonlines
from tqdm import tqdm
from copy import copy
import os
import json
import requests
import ujson as json
from openai import OpenAI
import re

os.environ["OPENAI_API_KEY"] = "e0a7d609-7a9d-401e-9cd7-53a3819a21f3"
os.environ["OPENAI_API_BASE"] = "http://gpt-proxy.jd.com/gateway/azure"

In [25]:
def open_ai_sdk(content):
    client = OpenAI(
        api_key=os.environ["OPENAI_API_KEY"],
        base_url=os.environ["OPENAI_API_BASE"],
    )
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}"
    }

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "assistant"}, 
                  {"content": content, "role": "user"}],
        temperature=0.5,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stream=False,
        # extra_body={"erp": "python"}
        # extra_headers=headers
    )

    return response

In [26]:
data = []
# 打开 JSON Lines 文件
with jsonlines.open('data/Preliminary_TestSet/Preliminary_TestSet.json') as reader:
    for obj in reader:
        # 处理 obj
        data.append(obj)
data = data[0]

In [ ]:

# 假设 data 是你要处理的数据列表

# 正则表达式模式
pattern = r"answer:\s*\w+"

no_matches_question = []


for obj in tqdm(data[:500], desc="Processing"):
    tmp_transfer_data = copy(obj)
    content = f"""
下面是的一道图分析相关的题目，可能是一道判断题或计算题。请你生成可以解出这道题目的python代码。下面是问题：{obj['question']}
"""
    python_code_response = open_ai_sdk(content)
    python_code=python_code_response.choices[0].message.content
    content = f"""
下面是的一道图分析相关的题目以及对应能解题的python代码，请执行代码后作答，如果你觉得这是一道判断题且结果为真，请返回TRUE，反之则返回FALSE。如果你觉得这是计算题，请返回带有两位小数的数值答案。
务必记住：1.你的回答不要有答案之外的任何字符！只需要最终答案！2.题目一定是判断题或计算题，不会有其他类型。
问题如下：{obj['question']}
python_代码：{python_code}
"""
    response = open_ai_sdk(content)
    
    result_string=response.choices[0].message.content
    obj['answer'] = result_string
    # result_string_list = result_string.split('\n')
    # # 使用正则表达式提取 'output: x' 中的 x
    # match = re.search(r'output:\s*(\S+)', result_string_list[-1])
    # if match:
    #     obj['answer'] = match.group(1)
    # else:
    #     obj['answer'] = ''
    #     no_matches_question.append(obj['id'])
            


    

Processing:  10%|██▉                         | 52/500 [15:38<2:06:00, 16.88s/it]

In [ ]:
# # 将'id'替换为'ID'
# for item in data:
#     item['ID'] = item.pop('id')
#     item['question'] = item.pop('question')
#     item['answer'] = item.pop('answer')

In [ ]:
import json



# 将列表保存为JSON格式的文件，每个元素占用一行
with open('data/res/大模型图图_result1.json', 'w', encoding='utf-8') as file:
    file.write('[\n')
    for i, item in enumerate(data):
        json_line = json.dumps(item, ensure_ascii=False)
        if i < len(data) - 1:
            file.write(f'{json_line},\n')
        else:
            file.write(f'{json_line}\n')
    file.write(']')